In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"

Running local


In [39]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Zamuro_DataLoader import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [40]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_wav_filenames( path_to_dir, suffix=".wav" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

In [43]:
files_Ais = find_csv_filenames(f"{root}/Zamuro/AI_Caract")
file_rain = f"{root}/Zamuro/Complementary_Files/Result_rain_merged.xlsx"
files_udas = f"{root}/Zamuro/Complementary_Files/UDAS Pasivo_20221001_Zamuro.xlsx"
files_covers = f"{root}/Zamuro/Complementary_Files/ReyZamuro_softLabels.csv"
files_recorders = os.listdir("/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/")
files_satelites = f"{root}/Zamuro/Complementary_Files/zamuro_satelites.csv"
path_recorders = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

In [75]:
UDAS = pd.read_excel(f"{files_udas}", sheet_name="Template")
covers = pd.read_csv(f"{files_covers}")
satelites = pd.read_csv(f"{files_satelites}")
satelites.rename(columns={"site":"field_number_PR"}, inplace=True)
# AIs = pd.read_csv(files_Ais)
habitat = ["bosque"]* 2 + ["pastizal"] + ["bosque"] + ["borde"] * 2 + ["bosque"] +  ["parche"] *2 + ["bosque"]*4 + ["borde"]*11 + ["bosque"] * 9 + ["parche"]*2 + ["pastizal"]*4 + ["parche"]*2 + ["sabana"]*10 + ["borde"] + ["sabana"] + ["pastizal"] + ["sabana"] + ["bosque"]*3 + ["pastizal"]*2 + ["potrero"] * 2 + ["bosque"] + ["borde"]* 5 +["bosque"] * 9 + ["sabana"]* 2 + ["pastizal"] + ["parche"]
habitat = [
    "bosque", "bosque", "pastizal", "borde", "borde", "borde", "bosque", "sabana", "bosque", "borde",
    "bosque", "bosque", "bosque", "bosque", "bosque", "borde", "borde", "pastizal", "potrero", "sabana",
    "borde", "bosque", "bosque", "sabana", "borde", "bosque", "borde", "bosque", "bosque", "bosque", "borde",
    "potrero", "bosque", "bosque", "sabana", "bosque", "bosque", "sabana", "parche", "bosque", "pastizal",
    "bosque", "bosque", "borde", "bosque", "borde", "borde", "bosque", "parche", "bosque", "bosque", "sabana",
    "bosque", "pastizal", "pastizal", "sabana", "sabana", "borde", "bosque", "sabana", "bosque", "sabana",
    "bosque", "sabana", "borde", "borde", "borde", "borde", "pastizal", "bosque", "pastizal", "pastizal",
    "bosque", "bosque", "bosque", "bosque", "sabana", "pastizal", "parche", "borde", "sabana", "sabana", "parche",
    "parche", "sabana", "borde", "sabana", "parche", "bosque", "sabana", "parche", "bosque", "sabana", "borde",
    "borde", "sabana", "sabana", "borde", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque", "bosque"
]

keep_cols = ["field_number_PR", "longitud_IG", "latitude_IG", "day_init_TI", "day_end_TI", "samp_rate_RE"]

In [6]:
UDAS = UDAS[keep_cols]
UDAS["habitat"] = habitat
UDAS.info()
covers.rename(columns={"field_numb":"field_number_PR", "1":"pasture", "2": "savanna", "3": "forest"}, inplace=True)
a=UDAS
b=covers
merged_df = a.merge(b, on='field_number_PR', how='left')
result_df = merged_df[merged_df['pasture'].notna()]
result_df.reset_index(inplace=True)
result_df.drop("index", axis=1, inplace=True)
result_df["cover"] = result_df[["pasture", "savanna", "forest"]].idxmax(axis=1)
result_df.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_recorders.csv")
result_df_satelites = result_df.merge(satelites, on='field_number_PR', how='left')
result_df_satelites.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_recorders_satelites.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   field_number_PR  106 non-null    object 
 1   longitud_IG      106 non-null    float64
 2   latitude_IG      106 non-null    float64
 3   day_init_TI      106 non-null    int64  
 4   day_end_TI       106 non-null    int64  
 5   samp_rate_RE     106 non-null    int64  
 6   habitat          106 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 5.9+ KB


/tmp/ipykernel_5308/3762597280.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.drop("index", axis=1, inplace=True)
/tmp/ipykernel_5308/3762597280.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["cover"] = result_df[["pasture", "savanna", "forest"]].idxmax(axis=1)


In [7]:
audios = pd.DataFrame(columns=result_df.columns)
total_files=0
for i in range(len(result_df)):
    aux = pd.DataFrame(columns=result_df.columns)
    path = root_path+"/"+result_df.loc[i, "field_number_PR"]
    files = os.listdir(path)
    total_files+=len(files)
    aux["Filename"] = files
    name=[i.split('.', 1)[0] for i in files]
    aux["name_FI"] = name
    for col in aux.columns[0:-2]:
        aux[col] = result_df.loc[i,col]
    audios = pd.concat([audios, aux])
audios.to_csv(f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv")
audios.reset_index(inplace=True)

In [8]:
rain = pd.read_excel(file_rain)
rain = rain[["field_number_PR", "name_FI","rain_FI"]]
audios["rain_FI"] = "-"
count = 0
for i in range(len(audios)):
    if i%10000 == 0:
        print(i)
    try:
        audio = audios.loc[i]
        audios.loc[i, "rain_FI"] = rain.loc[rain["field_number_PR"] == audio["field_number_PR"]].loc[rain["name_FI"] == audio["Filename"]]["rain_FI"].values[0]
    except:
        count+=1
audios.to_csv(f"{root}//Zamuro/Complementary_Files/zamuro_audios.csv")

0
10000
20000
30000
40000
50000
60000
70000


In [ ]:
lista = list(set(audios_zamuro["field_number_PR"]))
lista.sort()
for csv in lista:
    temporal_csv = audios_zamuro.loc[audios_zamuro["field_number_PR"] == csv]
    temporal_csv.drop(columns=["index"],inplace=True)
    temporal_csv.to_csv(f"{root}//Zamuro/Complementary_Files/{csv}.csv",
                        index=True, index_label="global_index")

In [1]:
audios_zamuro = audios
lista = list(set(audios_zamuro["field_number_PR"]))
lista.sort()
model = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"

NameError: name 'audios' is not defined

In [35]:
X = torch.load(f"temporal_zamuro/Features/AE_features_{date_format}_rain_NO_{lista[0]}.csv.pth",  map_location=torch.device('cpu'))
y = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{lista[0]}.csv.pth",  map_location=torch.device('cpu'))
for i, csv in enumerate(lista[1::]):
    X_i = torch.load(f"temporal_zamuro/Features/AE_features_{date_format}_rain_NO_{csv}.csv.pth",  map_location=torch.device('cpu'))
    y_i = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{csv}.csv.pth",  map_location=torch.device('cpu'))
    X = torch.cat((X,X_i),0)
    y = y + y_i
torch.save(X, f"temporal_zamuro/Features/AE_features_Zamuro.pth")
torch.save(y, f"temporal_zamuro/Features/AE_test_path_samples_Zamuro.pth")
       
path_error = torch.load(f"temporal_zamuro/Features/AE_test_corrupted_samples_list_{date_format}_rain_NO_{lista[9]}.csv.pth",  map_location=torch.device('cpu'))
y_path = torch.load(f"temporal_zamuro/Features/AE_test_path_samples_{date_format}_rain_NO_{lista[9]}.csv.pth",  map_location=torch.device('cpu'))

## PRUEBA - ELIMINAR CUANDO TERMINE -AÚN EN PRUEBA, ELIMINAR SI CLUSTERING FUNCIONA

In [31]:
lista = list(set(audios_zamuro["field_number_PR"]))
lista.sort()
model = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"
y = torch.load(f"temporal_zamuro/Features/AE_labels_{date_format}_rain_NO_{lista[0]}.csv.pth",  map_location=torch.device('cpu'))
recorder_list = []
hour_list = []
minute_list = []
for i, csv in enumerate(lista[1::]):
    y_i = torch.load(f"temporal_zamuro/Features/AE_labels_{date_format}_rain_NO_{csv}.csv.pth",  map_location=torch.device('cpu'))
    y = y + y_i
torch.save(y, f"temporal_zamuro/Features/AE_labels_Zamuro.pth")

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [215]:
path_flat = [item for sublist in y_path for item in sublist]
path_flat = np.asarray(path_flat)

### Acoustic Indices Organization

In [73]:
a = pd.read_pickle(f"{root}/Zamuro/AI_Caract/RZUA01_AIs.pickle")
split = a["file"][0].split("/")[-2::]
split

['RZUA01', '20220903_104500.WAV']

In [101]:
files_Ais = find_csv_filenames(f"{root}/Zamuro/AI_Caract")

for index, file in enumerate(files_Ais):
    AI = pd.read_csv(f"{root}/Zamuro/AI_Caract/{files_Ais[index]}")
    print(len(AI))
    for i in range(len(AI)):
        split = AI["file"][i].split("/")[-2::]
        AI["file"][i] = f"{split[0]}_{split[1]}"
    if index == 0:
        AIs_Zamuro = AI.copy()
    else:
        AIs_Zamuro = pd.concat([AIs_Zamuro, AI])
AIs_Zamuro.to_csv(f"{root}/Zamuro/AI_Caract/AI_Zamuro.csv")

1042
678
7
573


/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A

668
779
778


/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"


775
858


...

/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"


773
770


/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"


770
768


/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
/tmp/ipykernel_14232/3041969744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI["file"][i] = f"{split[0]}_{split[1]}"
